## CRF model

Modelo CRF para ato de aposentadoria (com dados rotulados manualmente)

In [37]:
# !pip install sklearn_crfsuite
import pandas as pd
import sklearn_crfsuite

In [38]:
# tokens = '/home/thais/Documentos/Knedle/experiments/members/jose_reinaldo/TCDF_aposentadoria/raw_tokenized_atos.parquet'
# texts = '/home/thais/Documentos/Knedle/experiments/members/jose_reinaldo/TCDF_aposentadoria/labelbox_texts_atos.parquet'
# tokens tratados
# texts = "/home/thais/Documentos/Knedle/experiments/members/thais/contrato-licitações/estudo_crf/result/labelbox_atos.parquet"
# tokens = "/home/thais/Documentos/Knedle/experiments/members/thais/contrato-licitações/estudo_crf/result/tokenized_texts.parquet"

# df1 = pd.read_parquet(tokens)
# df2 = pd.read_parquet(texts)

# result = pd.merge(df1, df2, on=["arquivo_rast"])
# result.to_csv('result.csv')
# result.head()

In [39]:
df= pd.read_csv('result.csv')
result = pd.DataFrame(df)
result.tokenized_text = result.tokenized_text.apply(lambda x: x.encode('ascii','ignore'))

Load csv with text and labels

In [40]:
x = []
y = []
for row in range(len(result)):
    if pd.notna(result['treated_text'][row]):
        x.append(result['treated_text'][row].split())
        y.append(result['tokenized_text'][row].split())
    # if pd.notna(result['tokenized_text'][row]).any():
    #     y.append(result['tokenized_text'][row])
len(x), len(y)

(1168, 1168)

In [41]:
df = pd.DataFrame(result)
df.head(3)

,Unnamed: 0,arquivo_rast,tokenized_text,text,ato,dodf,treated_text
0,0,99_extrato_de_contrato_ou_convenio_DODF_179_17...,"b""[' O', ' O', ' O', ' O', ' B-NUM_AJUSTE', ' ...",xxbcet EXTRATO DO CONTRATO Nº 292/2010. xxecet...,extrato_de_contrato_ou_convenio,DODF_179_17-09-2010,EXTRATO DO CONTRATO Nº 292/2010. Processo: 150...
1,1,97_extrato_de_contrato_ou_convenio_DODF_194_10...,"b""[' O', ' O', ' O', ' O', ' B-NUM_AJUSTE', ' ...",xxbcet EXTRATO DO CONTRATO Nº 06/2019 xxecet\n...,extrato_de_contrato_ou_convenio,DODF_194_10-10-2019,EXTRATO DO CONTRATO Nº 06/2019 FORNCIMENTO DE ...
2,2,944_extrato_de_contrato_ou_convenio_DODF_237_1...,"b""[' O', ' O', ' O', ' O', ' O', ' O', ' O', '...",xxbcet EXTRATO DE CONTRATO xxecet\r\r\n1) Cont...,extrato_de_contrato_ou_convenio,DODF_237_12-11-2013,EXTRATO DE CONTRATO 1) Contrato n.º 22/2013 TC...


In [42]:
y[0]

[b"['",
 b"O',",
 b"'",
 b"O',",
 b"'",
 b"O',",
 b"'",
 b"O',",
 b"'",
 b"B-NUM_AJUSTE',",
 b"'",
 b"O',",
 b"'",
 b"O',",
 b"'",
 b"O',",
 b"'",
 b"B-PROCESSO',",
 b"'",
 b"O',",
 b"'",
 b"O',",
 b"'",
 b"O',",
 b"'",
 b"B-CONTRATANTE',",
 b"'",
 b"O',",
 b"'",
 b"B-CONTRATADA',",
 b"'",
 b"I-CONTRATADA',",
 b"'",
 b"I-CONTRATADA',",
 b"'",
 b"O',",
 b"'",
 b"O',",
 b"'",
 b"O',",
 b"'",
 b"O',",
 b"'",
 b"O',",
 b"'",
 b"O',",
 b"'",
 b"O',",
 b"'",
 b"O',",
 b"'",
 b"O',",
 b"'",
 b"O',",
 b"'",
 b"O',",
 b"'",
 b"O',",
 b"'",
 b"O',",
 b"'",
 b"O',",
 b"'",
 b"B-OBJ_AJUSTE',",
 b"'",
 b"I-OBJ_AJUSTE',",
 b"'",
 b"I-OBJ_AJUSTE',",
 b"'",
 b"I-OBJ_AJUSTE',",
 b"'",
 b"I-OBJ_AJUSTE',",
 b"'",
 b"I-OBJ_AJUSTE',",
 b"'",
 b"I-OBJ_AJUSTE',",
 b"'",
 b"I-OBJ_AJUSTE',",
 b"'",
 b"I-OBJ_AJUSTE',",
 b"'",
 b"I-OBJ_AJUSTE',",
 b"'",
 b"I-OBJ_AJUSTE',",
 b"'",
 b"I-OBJ_AJUSTE',",
 b"'",
 b"I-OBJ_AJUSTE',",
 b"'",
 b"I-OBJ_AJUSTE',",
 b"'",
 b"I-OBJ_AJUSTE',",
 b"'",
 b"I-OBJ_AJUSTE',",
 b"'",

Create dictionary feature for each word in each sequence in x

In [43]:
def get_features(sentence):
        """Create features for each word in act.
        Create a list of dict of words features to be used in the predictor module.
        Args:
            act (list): List of words in an act.
        Returns:
            A list with a dictionary of features for each of the words.
        """
        sent_features = []
        for i in range(len(sentence)):
            # print(sentence[i])
            word_feat = {
                'word': sentence[i].lower(),
                'capital_letter': sentence[i][0].isupper(),
                'all_capital': sentence[i].isupper(),
                'isdigit': sentence[i].isdigit(),
                'word_before': sentence[i].lower() if i == 0 else sentence[i-1].lower(),
                'word_after:': sentence[i].lower() if i+1 >= len(sentence) else sentence[i+1].lower(),
                'BOS': i == 0,
                'EOS': i == len(sentence)-1
            }
            sent_features.append(word_feat)
        return sent_features
    
for i in range(len(x)):
    x[i] = get_features(x[i])

In [44]:
x[0]

[{'word': 'extrato',
  'capital_letter': True,
  'all_capital': True,
  'isdigit': False,
  'word_before': 'extrato',
  'word_after:': 'do',
  'BOS': True,
  'EOS': False},
 {'word': 'do',
  'capital_letter': True,
  'all_capital': True,
  'isdigit': False,
  'word_before': 'extrato',
  'word_after:': 'contrato',
  'BOS': False,
  'EOS': False},
 {'word': 'contrato',
  'capital_letter': True,
  'all_capital': True,
  'isdigit': False,
  'word_before': 'do',
  'word_after:': 'nº',
  'BOS': False,
  'EOS': False},
 {'word': 'nº',
  'capital_letter': True,
  'all_capital': False,
  'isdigit': False,
  'word_before': 'contrato',
  'word_after:': '292/2010.',
  'BOS': False,
  'EOS': False},
 {'word': '292/2010.',
  'capital_letter': False,
  'all_capital': False,
  'isdigit': False,
  'word_before': 'nº',
  'word_after:': 'processo:',
  'BOS': False,
  'EOS': False},
 {'word': 'processo:',
  'capital_letter': True,
  'all_capital': False,
  'isdigit': False,
  'word_before': '292/2010.',
 

Separate train and test splits (in order)

In [45]:
import math
split_idx = math.floor(0.8*len(x))
x_train = x[0:split_idx]
y_train = y[0:split_idx]
x_test = x[split_idx:]
y_test = y[split_idx:]
len(x_train),  len(x_test), len(y_train)


(934, 234, 934)

In [46]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics

crf = sklearn_crfsuite.CRF(
    algorithm = 'l2sgd',
    c2=1,
    max_iterations=100,
    all_possible_transitions=True
)

crf.fit(x_train, y_train)

ValueError: The numbers of items and labels differ: |x| = 159, |y| = 384

In [ ]:
classes = list(crf.classes_)
classes.remove('O')

y_pred = crf.predict(x_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=classes)

/home/jose/anaconda3/envs/ner/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


0.5114313469557288

In [ ]:
print(metrics.flat_classification_report(
    y_test, y_pred, labels=classes, digits=3
))

/home/jose/anaconda3/envs/ner/lib/python3.8/site-packages/sklearn/utils/validation.py:68: FutureWarning: Pass labels=['B-QUADRO', 'I-QUADRO', 'B-ATO', 'I-ATO', 'B-NOME_ATO', 'I-NOME_ATO', 'B-CARGO', 'I-CARGO', 'B-CLASSE', 'I-CLASSE', 'B-PADRAO', 'I-PADRAO', 'B-FUND_LEGAL', 'I-FUND_LEGAL', 'B-EMPRESA_ATO', 'I-EMPRESA_ATO', 'B-PROCESSO', 'I-PROCESSO', 'B-COD_MATRICULA_ATO', 'I-COD_MATRICULA_ATO'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/home/jose/anaconda3/envs/ner/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jose/anaconda3/envs/ner/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetri

                     precision    recall  f1-score   support

           B-QUADRO      0.240     0.006     0.012      1017
           I-QUADRO      0.373     0.008     0.015      5348
              B-ATO      0.800     1.000     0.889       963
              I-ATO      0.805     1.000     0.892       808
         B-NOME_ATO      0.678     0.965     0.796      1032
         I-NOME_ATO      0.675     0.970     0.796      3290
            B-CARGO      0.464     0.364     0.408      1030
            I-CARGO      0.447     0.382     0.412      3202
           B-CLASSE      0.522     0.519     0.521       520
           I-CLASSE      0.522     0.517     0.520       522
           B-PADRAO      0.496     0.359     0.417      1008
           I-PADRAO      0.496     0.359     0.417      1008
       B-FUND_LEGAL      0.770     0.661     0.711       386
       I-FUND_LEGAL      0.741     0.625     0.678     11720
      B-EMPRESA_ATO      0.350     0.069     0.115       508
      I-EMPRESA_ATO    